In [1]:
def gini_impurity(y):
    # calculate gini_impurity given labels/classes of each example
    m = y.shape[0]
    cnts = dict(zip(*np.unique(y, return_counts = True)))
    impurity = 1 - sum((cnt/m)**2 for cnt in cnts.values())
    return impurity

def entropy(y):
    # calculate entropy given labels/classes of each example
    m = y.shape[0]
    cnts = dict(zip(*np.unique(y, return_counts = True)))
    disorder = - sum((cnt/m)*log(cnt/m) for cnt in cnts.values())
    return disorder

In [3]:
def get_split(X, y):
    # loop through features and values to find best combination with the most information gain
    best_gain, best_index, best_value = 0, None, None
    cur_gini = gini_impurity(y)
    n_features = X.shape[1]  

    for index in range(n_features):  
        values = np.unique(X[:, index], return_counts = False)  

        for value in values:  
            left, right = test_split(index, value, X, y)
            if left['y'].shape[0] == 0 or right['y'].shape[0] == 0:
                continue
            gain = info_gain(left['y'], right['y'], cur_gini)
            if gain > best_gain:
                best_gain, best_index, best_value = gain, index, value
    best_split = {'gain': best_gain, 'index': best_index, 'value': best_value}
    return best_split
    
def test_split(index, value, X, y):
    # split a group of examples based on given index (feature) and value
    mask = X[:, index] < value 
    left = {'X': X[mask, :], 'y': y[mask]}
    right = {'X': X[~mask, :], 'y': y[~mask]}
    return left, right
    
def info_gain(l_y, r_y, cur_gini):
    # calculate the information gain for a certain split
    m, n = l_y.shape[0], r_y.shape[0]
    p = m / (m + n)
    return cur_gini - p * gini_impurity(l_y) - (1 - p) * gini_impurity(r_y)

In [ ]:
class Decision_Node:
    # define a decision node
    def __init__(self, index, value, left, right):
        self.index, self.value = index, value
        self.left, self.right = left, right

class Leaf:
    # define a leaf node
    def __init__(self, y):
        self.counts = dict(zip(*np.unique(y, return_counts = True)))
        self.prediction = max(self.counts.keys(), key = lambda x: self.counts[x])